In [1]:
from data_service import get_sales_depart_billboard
import pandas as pd

In [3]:
# 数据获取
billboard_df: pd.DataFrame = get_sales_depart_billboard(
    '2013-01-01', '2021-12-31')

# 数据储存
billboard_df.to_csv('../data/billboard.csv', encoding='utf-8')

机构龙虎榜数据获取:   0%|          | 0/2188 [00:00<?, ?it/s]

In [3]:
import os

os.listdir('..')

['data', 'data_service', 'scr', '特征分布择时.ipynb']

In [4]:
# 读取本地文件
billboard_df: pd.DataFrame = pd.read_csv('../data/billboard.csv',
                                         encoding='utf-8',
                                         index_col=[0],
                                         parse_dates=True)


In [5]:
billboard_df

,trade_date,ts_code,exalter,buy,buy_rate,sell,sell_rate,net_buy,side,reason
0,2013-01-04,000043.SZ,中国国际金融有限公司上海淮海中路证券营业部,16970815.06,1.19,16377628.00,1.24,-593187.06,0,连续三个交易日内，涨幅偏离值累计达20%的证券
1,2013-01-04,000043.SZ,中国国际金融有限公司上海淮海中路证券营业部,16970815.06,1.19,16377628.00,1.24,-593187.06,1,连续三个交易日内，涨幅偏离值累计达20%的证券
2,2013-01-04,000043.SZ,中国国际金融有限公司深圳福华一路证券营业部,14052271.14,0.00,0.00,1.02,-14052271.14,1,连续三个交易日内，涨幅偏离值累计达20%的证券
3,2013-01-04,000043.SZ,中国银河证券股份有限公司绍兴证券营业部,5276763.48,1.13,15543450.07,0.38,10266686.59,0,连续三个交易日内，涨幅偏离值累计达20%的证券
4,2013-01-04,000043.SZ,光大证券股份有限公司深圳金田路证券营业部,3067442.00,2.02,27633247.70,0.22,24565805.70,0,连续三个交易日内，涨幅偏离值累计达20%的证券
...,...,...,...,...,...,...,...,...,...,...
683,2021-12-31,900943.SH,国泰君安证券股份有限公司国际部,NaN,NaN,126353.40,10.16,-126353.40,1,涨幅偏离值达7%的证券
684,2021-12-31,900943.SH,招商证券股份有限公司北京安立路证券营业部,NaN,NaN,74719.20,6.01,-74719.20,1,涨幅偏离值达7%的证券
685,2021-12-31,900943.SH,中国中金财富证券有限公司武汉中北路证券营业部,NaN,NaN,57048.50,4.59,-57048.50,1,涨幅偏离值达7%的证券
686,2021-12-31,900943.SH,第一创业证券股份有限公司杭州来福士营业部,NaN,NaN,56658.90,4.56,-56658.90,1,涨幅偏离值达7%的证券
